# Pythonで学ぶ強化学習　ハンズオン

## Day1 （強化学習の位置付けを知る）

### 1. 強化学習は学習方法の一種（抜粋）
- 教師有り学習
    - データと正解ラベルをセットで与える。　データが与えられたらラベルが出力されるようパラメータを調整。
- 教師なし学習
    - データのみを与えてデータの特徴（構造や表現）を抽出できるようパラメータを調整。
- 強化学種
    - 行動により報酬が得られる環境（タスク）を与えて、各状態で報酬に繋がる行動が出力される行動が出力されるようモデルのパラメータを調整。

### 2. 強化学習における問題設定: Markov Desiosion Process

- 強化学習における「環境」は以下の要素からなる**マルコフ決定過程（Markov Desiosion Process）**からなる。
    - $s$ : **状態 （state）**
    - $a$ : **行動 （action）**
    - $T$ : 状態遷移の確率（**遷移関数 / Transition function**）。状態$s$と行動$a$を引数に、遷移先$s^{\prime}$（次の状態）と遷移確率を出力する関数。
    - $R$ : **報酬関数 （Reward function）**。状態$s$と遷移先$s^{\prime}$を引数に、報酬を出力する関数（行動$a$を引数に取ることもある）。
    

- **エピソード （Episode）**: 環境の開始から終了までの期間。

- **戦略** $\pi$: 状態$s$を受け取り、行動$a$を決める関数。

- **エージェント**: 戦略$\pi$に従って行動する主体。（強化学習では戦略がモデルとなり、パラメーターを調節して適切な行動を出力する）

- **即時報酬 （Immediate reward）**: MDPにおける報酬 $r$（$:= R(s, s^{\prime})$）。

- **期待報酬（Expected reward）** / **価値（Value）**: 報酬の総和

- **価値評価（Value apploximation）**: 価値を算出すること。

MDPにおける時刻$t$ から時刻$T$ までの「報酬の総和」は**割引率（Discount factor）** $\gamma$を用いて以下のように書ける。

$$G_{t} := r_{t+1} + \gamma r_{t+2} + \gamma^{2}r_{t+2} + \cdots + \gamma^{T-t-1}r_{tT} = \sum_{k=0}^{T-t+1}\gamma^{k}r_{t+k+1}$$


価値$G_{t}$は時刻$t+1$での価値を用いて、再帰的に以下のように書ける。

$$G_{t} = r_{t+1} + \gamma G_{t+1}$$

### 3. Example(迷路)

簡単な迷路における、MDPの構成要素は以下である。

- $s$ 状態 : セルの位置（行/列）
- $a$ 行動 : 上下左右への移動
- $T$ 遷移関数: 状態と行動を受け取り、移動可能なセルとそこへの遷移確率を返す関数
- $R$ 報酬関数: 状態を受け取り、緑(ゴール)のセルなら1、赤(ペナルティ)のセルなら-1を返す関数

    

1. 位置クラス(状態$s$)、上下移動クラス（行動$a$）の実装

In [1]:
from enum import Enum

# セル位置表現クラス
class State:
    def __init__(self, row: int=-1, column: int=-1):
        self.row = row
        self.column = column
    
    def __repr__(self):
        return f"State: [{self.row}, {self.column}]"
    
    def clone(self):
        return State(self.row, self.column)
    
    def __hash__(self):
        return hash((self.row, self.column))
    
    def __eq__(self, other):
        return self.row == other.row and self.column == other.column
    
# アクション定義クラス
class Action(Enum):
    UP = 1
    DOWN = -1
    LEFT = 2
    RIGHT = -2

In [2]:
State(3, 5)

State: [3, 5]

In [3]:
Action.UP

<Action.UP: 1>

In [4]:
Action(1)

<Action.UP: 1>

2. 環境実態クラス(雛形)を実装

In [5]:
from typing import List, Dict

# 迷路環境実態クラス
class Environment:
    def __init__(self, grid: List[List[int]], move_prob: float=0):
        """
        エージェントがゴールを早く目指すように、デフォルトの報酬を負値に定義。
        エージェントは選択した向きにmove_probの確率で移動し、(1 - move_prob)の確率で他の方向に進む。
        """
        # grid is 2d-Array. Its values are treated as an attribute.
        # kinds of attributes is following
        # 0: ordinary cell
        # -1: damage cell (game end)
        # 1: reward cepp (game end)
        # 9: block cell (can't locate agent)
        self.grid = grid
        self.agent_state = State()
        self.default_reward = -0.04
        self.move_prob = move_prob
        self.reset()
        
    @property
    def row_length(self) -> int:
        return len(self.grid)
    
    @property
    def column_length(self) -> int:
        return len(self.grid[0])
    
    @property
    def actions(self) -> List[Action]:
        return [Action.UP, Action.DOWN, Action.LEFT, Action.RIGHT]
    
    @property
    def states(self) -> List[State]:
        """
        迷路内の移動可能なセルを返す。(ブロックセルを除外)
        """
        states = []
        for row in range(self.row_length):
            for col in range(self.column_lengthh):
                if self.grid[row][col] != 9:
                    states.append(State(row, col))
        return states

3. 遷移関数$T$と報酬関数$R$の実装

In [6]:
class Environment(Environment):
    def transition_func(self, state: State, action: Action) -> Dict[State, float]:
        """
        状態とアクションを受けとり、次の状態への遷移確率を返す。
        今回の迷路では、move_probの確率で選択した方向に、(1-move_prob)の確率で、選択した方向との反対以外の方向に等確率で遷移する。
        """
        transition_probs = {}
        for suggest_action in self.actions:
            next_state = self._move(state, suggest_action)
            
            if suggest_action == action:
                prob = move_prob
            elif suggest_action.value + action.value != 0:
                prob = (1 - move_prob) / 2
            else:
                prob = 0
                
            transition_probs[next_state] = prob
        
        return transition_probs
    
    def can_action_at(self, state: State) -> bool:
        """
        stateがアクション可能なセルかどうか判定
        """
        try:
            if self.grid[state.row][state.column] == 0:
                True
            else:
                False
        except IndexError as e:
            raise ValueError(f"This state is out of mase! {state}")
    
    def _move(self, state: State, aciton) -> State:
        """
        位置とアクションを受け取り、アクション可能な位置であれば受け取ったアクション方向に移動した位置に移動する。
        移動先位置が迷路の外であれば、そのままの位置を返す。
        """
        if not self.can_action_at:
            raise ValueError("Can't move from here!")
        
        next_state = state.clone()
    